In [ ]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import time

import src.utils.las_utils as las_utils
import src.utils.csv_utils as csv_utils
import src.utils.ahn_utils as ahn_utils
import src.fusion as fusion
from src.labels import Labels
from src.pipeline import Pipeline

## Street furniture fuser based on cluster dimensions and BGT reference data
First, download and parse the required BGT data in notebook [1. Generate reference data](1.%20Generate%20reference%20data.ipynb).

In [ ]:
# The already labelled points must be removed.
exclude_labels = (Labels.GROUND, Labels.BUILDING, Labels.TREE, Labels.STREET_LIGHT, Labels.TRAFFIC_SIGN, Labels.CAR, Labels.NOISE)
# TODO

In [ ]:
# Select the file to process.
tilecode = '2386_9702'

in_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'
out_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'

# Data folder for the fusers.
bgt_data_file = '../datasets/bgt/bgt_street_furniture_points_demo.csv'

# We need elevation data to determine object height above ground.
ahn_data_folder = '../datasets/ahn/'
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

In [ ]:
# Object type-specific parameters.
trash_can_params = {'min_height': 0.5, 'max_height': 1.5, 'min_width': 0.2, 'max_width': 1.5, 'min_length': 0.2, 'max_length': 1.5}
city_bench_params = {'min_height': 0.5, 'max_height': 1.5, 'min_width': 0.2, 'max_width': 2.0, 'min_length': 0.2, 'max_length': 3.5}

# Create fusers for BGT point data.
trash_can_fuser = fusion.BGTStreetFurnitureFuser(Labels.TRASH_CAN, bgt_type='afvalbak', ahn_reader=ahn_reader, bgt_file=bgt_data_file,
                                                 max_dist=1., params=trash_can_params)
city_bench_fuser = fusion.BGTStreetFurnitureFuser(Labels.CITY_BENCH, bgt_type='bank', ahn_reader=ahn_reader, bgt_file=bgt_data_file,
                                                  max_dist=1.2, params=city_bench_params)

## Process point clouds and save the results
The pipeline can now be used to process point clouds. Labels will be written to the LAS file as a new extended field called label.

In [ ]:
# Set-up the order in how we want to label.
process_sequence = (city_bench_fuser,) # TODO worden de masks doorgegeven
pipeline = Pipeline(processors=process_sequence, exclude_labels=exclude_labels, caching=False)

## Process a single file

In [ ]:
# Process the file.
start = time.time()
pipeline.process_file(in_file, out_file=out_file)
end = time.time()
print(f'Tile labelled in {end-start:.2f} seconds.')